# View pcigale fit result


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024-02-06
- uddate : 2024-02-06

Find pcigale here https://cigale.lam.fr



- adaptation : Sylvie Dagoret-Campagne from  https://cigale.lam.fr


In [ ]:
#%pylab widget
#import matplotlib.pyplot as plt
#matplotlib.rcParams['figure.figsize'] = [9.,6.]

In [ ]:
#%pylab widget
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [9.,6.]

In [ ]:
import numpy as np

from IPython.display import display
from ipywidgets import widgets
from matplotlib import pyplot as plt

from astropy.table import Table
from astropy.io import fits

from pcigale import sed
from pcigale import sed_modules as modules
from pcigale.warehouse import SedWarehouse
from pcigale.data import SimpleDatabase
import os,re

In [ ]:
path = "20240205_181701_out"

In [ ]:
def get_fitresults_dict(the_path):
    """
    paramters:
     - path : path where pcigale write results
    """
    all_files_runpcigale = sorted(os.listdir(the_path))
    spec_dict = {}
    for idx,filename in enumerate(all_files_runpcigale):

        # skip all file that are not ended by fits
        if not re.search('.*[.]fits$',filename):
            continue

        # 1) decode the spec number
        str_spec_found = re.findall('(^SPEC.+?)_.*', filename)
        #print("\t", idx," ",filename," ", str_spec_found)
        if len( str_spec_found) == 0:
            print(f"Skip filename {filename}")
            continue
        str_spec_found = str_spec_found[0]
  

        if str_spec_found in spec_dict.keys():
            #print("Found in dict ", spec_dict[str_spec_found])
            pass
        else:
            #print(f" ! {str_spec_found} NOT found in dict ")
            spec_dict[str_spec_found] = {"best_model": None,"SFH":None}

        d = spec_dict[str_spec_found]
    
        # 2) decode thefilename
        str_file_found = re.findall('^SPEC.+_(.*)[.]fits$', filename)[0]
        #print(idx,filename,str_file_found)

        if str_file_found == "SFH":
            d["SFH"] = filename
        elif str_file_found == "model":
            d["best_model"] = filename

        spec_dict[str_spec_found] = d
        
    return spec_dict
               
    

## Input

### Dictionary of file

In [ ]:
spec_dict = get_fitresults_dict(path)

### Select spectrum

In [ ]:
index = 10
spec_name = list(spec_dict.keys())[index]
spec_name

In [ ]:
spec_dict[spec_name]

In [ ]:
file_best_model = os.path.join(path,spec_dict[spec_name]['best_model'])
file_sfh = os.path.join(path,spec_dict[spec_name]['SFH'])

In [ ]:
t_best = Table.read(file_best_model)
t_sfh = Table.read(file_sfh)

In [ ]:
t_best[:5]

## Read all results

In [ ]:
t_res = Table.read(os.path.join(path,'results.fits'))

In [ ]:
t_res[:4]

### Selection 

In [ ]:
cut_select = t_res["id"] == spec_name

In [ ]:
t_res_sel = t_res[cut_select]
t_res_sel

In [ ]:
t_res_sel["best.reduced_chi_square"][0]

### Read data

In [ ]:
file_data = os.path.join(path,"observations.fits")
FORS2= Table.read(file_data)
FORS2.add_index('id')

In [ ]:
FORS2[:4]

### Select

In [ ]:
cut_fors2select = FORS2["id"] == spec_name

In [ ]:
FORS2_sel = FORS2[cut_fors2select]

In [ ]:
FORS2_sel

### Filters

In [ ]:
BANDS = [
    col for col in FORS2.colnames if not col.endswith("_err") 
    and not col.startswith("line") 
    and not col in ["id", "redshift", "distance"]
]

#with Database() as d:
#    BANDS_WAVE = np.array([d.get_filter(name).pivot_wavelength for name in BANDS])


with SimpleDatabase('filters') as d:
    #BANDS_WAVE = np.array([d.get(name).pivot_wavelength for name in BANDS])
    BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])
    

In [ ]:
#list_of_columns = list(t_res_sel.columns)
#for idx,col in enumerate(list_of_columns):
#    print(idx,col)

In [ ]:
# Initial values of the SED parameters
SED_PARAMETERS = {
    'sfhdelayed': {
        'tau_main': t_res_sel["best.sfh.tau_main"][0],
        'age_main': t_res_sel["best.sfh.age_main"][0],
        'tau_burst': t_res_sel["best.sfh.tau_burst"][0],
        'age_burst': t_res_sel["best.sfh.age_burst"][0],
        'f_burst': t_res_sel["best.sfh.f_burst"][0],
        },
    'bc03': {
        'imf': t_res_sel["best.stellar.imf"][0],
        'metallicity': t_res_sel["best.stellar.metallicity"][0],
    },
    'nebular': {
        'logU': t_res_sel["best.nebular.logU"][0],
        'f_esc': t_res_sel["best.nebular.f_esc"][0],
        'f_dust': t_res_sel["best.nebular.f_dust"][0],
        'emission': True,
    },
    'dustatt_modified_starburst': {
        'E_BV_lines': t_res_sel["best.attenuation.E_BV_lines"][0],
        'E_BV_factor': t_res_sel["best.attenuation.E_BV_factor"][0],
        'uv_bump_wavelength': t_res_sel["best.attenuation.uv_bump_wavelength"][0],
        'uv_bump_width': t_res_sel["best.attenuation.uv_bump_width"][0],
        'uv_bump_amplitude': t_res_sel["best.attenuation.uv_bump_amplitude"][0],
        'powerlaw_slope': t_res_sel["best.attenuation.powerlaw_slope"][0],
        'Ext_law_emission_lines': 1,
        'Rv': 3.1,
    },
    'dl2014': {
        'qpah': 2.50,
        'umin': 1.5,
        'alpha': 2.0,
        'gamma': 0.02,
    },
    'redshifting': {
        'redshift': t_res_sel["best.universe.redshift"][0],
    }
}

In [ ]:
WAREHOUSE = SedWarehouse()

def plot_sed(name, log_adjust_factor, tau_main, age_main, tau_burst, age_burst, f_burst, metallicity, logU,
             f_esc, f_dust, E_BV_lines, E_BV_factor, uv_bump_width, uv_bump_amplitude, powerlaw_slope, Rv, qpah, 
             umin, alpha, gamma, redshift, l_range):

    chi2_red = t_res_sel["best.reduced_chi_square"][0]
    redshift = t_res_sel["best.universe.redshift"][0]	
    
    plt.figure("Interactive_SED", figsize=(9, 7))
    sed = WAREHOUSE.get_sed(
        module_list = ['sfhdelayed', 'bc03', 'nebular', 'dustatt_modified_starburst', 'dl2014', 'redshifting'],
        parameter_list= [
            {
                'tau_main': tau_main,
                'age_main': age_main,
                'tau_burst': tau_burst,
                'age_burst': age_burst,
                'f_burst': f_burst,
            },{
                'imf': 0,
                'metallicity': metallicity,
            },{
                'logU': logU,
                'f_esc': f_esc,
                'f_dust': f_dust,
                'emission': True,
            },{
                'E_BV_lines': E_BV_lines,
                'E_BV_factor': E_BV_factor,
                'uv_bump_wavelength': 217.5,
                'uv_bump_width': uv_bump_width,
                'uv_bump_amplitude': uv_bump_amplitude,
                'powerlaw_slope': powerlaw_slope,
                'Ext_law_emission_lines': 1,
                'Rv': Rv,
            },{
                'qpah': qpah,
                'umin': umin,
                'alpha': alpha,
                'gamma': gamma,
            }, {
                'redshift': redshift,
            }
        ]
    )
    
    adjust_factor = 10**log_adjust_factor

    # the model
    band_sed_fluxes = np.array([adjust_factor * sed.compute_fnu(band) for band in BANDS])

    # the data
    band_gal_fluxes = np.array([FORS2.loc[name][band] for band in BANDS])
    band_gal_err = [FORS2.loc[name][f"{band}_err"] for band in BANDS]
    
    x_lims = (10**l_range[0], 10**l_range[1])
    plt.clf()
    plt.grid()
    plt.loglog(sed.wavelength_grid, adjust_factor * sed.fnu ,'b-' ,label="SED")
    plt.scatter(BANDS_WAVE, band_sed_fluxes, color="r",label="SED band fluxes")
    plt.errorbar(BANDS_WAVE, band_gal_fluxes, yerr=band_gal_err, fmt='o',color="k" ,label=f"{name} fluxes, chi2_red = {chi2_red:.3f}")
    plt.xlim(x_lims)
    # Recompute the y limits
    
    mask_sed = (sed.wavelength_grid >= x_lims[0]) & (sed.wavelength_grid <= x_lims[1])
    mask_bands = (BANDS_WAVE >= x_lims[0]) & (BANDS_WAVE <= x_lims[1])

    y1 = np.min(adjust_factor * sed.fnu[mask_sed])
    y2 = np.min(band_sed_fluxes[mask_bands])

    # handle flux not Nan
    data_fluxes = band_gal_fluxes[mask_bands]
    data_fluxes = data_fluxes[~np.isnan(data_fluxes)]
    y3 = np.min(data_fluxes)

    y4 = np.max(adjust_factor * sed.fnu[mask_sed])
    y5 = np.max(band_sed_fluxes[mask_bands])
    y6 = np.max(data_fluxes)

  
    y_min = 0.9 * np.min([y1,y2,y3])
    if y_min <= 0:
        y_min = 1e-40  # log axis
    y_max = 1.1 * np.max([y4,y5,y6])

    
    plt.ylim((y_min, y_max))
    plt.legend(loc=0)
    plt.ylabel ("Flux [Jy]")
    plt.xlabel("Wavelength [nm]")
    plt.tight_layout()
    plt.title(f"Fors2 Spectrum {name} , z = {redshift:.3f}, in pcigale run {path}")
    _ = plt.show()

In [ ]:
plt.close("Interactive_SED")  # needed to rerun the cell

name = widgets.Dropdown(value=FORS2_sel['id'][0], 
                        options=list(FORS2_sel['id']))

log_adjust_factor = widgets.FloatSlider(11, min=0, max=16, step=0.1)

tau_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_main'], 
                               min=500, max=8000, step=100, 
                               description="tau_main", continuous_update=False)
age_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_main'], 
                               min=2000, max=10000, step=1000, 
                               description="age_main", continuous_update=False)
tau_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_burst'], 
                                min=500, max=40000, step=100, 
                                description="tau_burst", continuous_update=False)
age_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_burst'],
                                min=50, max=500, step=100, 
                                description="age_burst", continuous_update=False)
f_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['f_burst'], 
                              min=0, max=.9, step=.1, 
                              description="f_burst", continuous_update=False)

metallicity = widgets.Dropdown(value=SED_PARAMETERS['bc03']['metallicity'], 
                               options=[0.0001, 0.0004, 0.004, 0.008, 0.02, 0.05], 
                               description="metallicity")

logU = widgets.Dropdown(value=SED_PARAMETERS['nebular']['logU'],
                        options=[-4.0, -3.9, -3.8, -3.7, -3.6, -3.5, -3.4, -3.3, 
                                 -3.2, -3.1, -3.0, -2.9, -2.8, -2.7, -2.6, -2.5, 
                                 -2.4, -2.3, -2.2, -2.1, -2.0, -1.9, -1.8, -1.7,
                                 -1.6, -1.5, -1.4, -1.3, -1.2, -1.1, -1.0],
                        description="logU")
f_esc = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_esc'], 
                            min=0, max=1, step=.1, 
                            description="f_esc", continuous_update=False)
f_dust = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_dust'], 
                             min=0, max=1, step=.1, 
                             description="f_dust", continuous_update=False)
# f_esc + f_dust is at most 1 (the remaining part is emitted as lines)
def update_f_esc(*args):
    if f_esc.value + f_dust.value > 1:
        f_esc.value = 1 - f_dust.value
f_dust.observe(update_f_esc, 'value')
def update_f_dust(*args):
    if f_esc.value + f_dust.value > 1:
        f_dust.value = 1 - f_esc.value
f_esc.observe(update_f_dust, 'value')

E_BV_lines = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_lines'],
                                 min=0, max=1, step=.05,
                                 description='E_BV_lines', continuous_update=False)
E_BV_factor = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_factor'],
                                 min=0, max=1, step=.1,
                                 description='E_BV_factor', continuous_update=False)
uv_bump_amplitude = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_amplitude'], 
                                        min=0, max=5, step=0.1, 
                                        description="bump_ampl.", continuous_update=False)
uv_bump_width = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_width'],
                                    min=100, max=500, step=100, 
                                    description="bump_width", continuous_update=False)

powerlaw_slope  = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['powerlaw_slope'],
                                      min=-1, max=1, step=.1, 
                                      description="slope", continuous_update=False)
Rv = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['Rv'],
                         min=0, max=5, step=.1, 
                         description="Rv", continuous_update=False)

qpah = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['qpah'],
                        options=[0.47, 1.12, 1.77, 2.50, 3.19, 3.90, 4.58, 5.26, 5.95, 
                                 6.63, 7.32],
                        description="qpah")
umin = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['umin'],
                        options=[0.100, 0.120, 0.150, 0.170, 0.200, 0.250, 0.300, 0.350, 
                                 0.400, 0.500, 0.600, 0.700, 0.800, 1.000, 1.200, 1.500, 
                                 1.700, 2.000, 2.500, 3.000, 3.500, 4.000, 5.000, 6.000, 
                                 7.000, 8.000, 10.00, 12.00, 15.00, 17.00, 20.00, 25.00, 
                                 30.00, 35.00, 40.00, 50.00],
                        description="umin")
alpha = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['alpha'],
                            min=1., max=3., step=.5, 
                            description="alpha", continuous_update=False)
gamma = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['gamma'],
                            min=0., max=20., step=1, 
                            description="gamma", continuous_update=False)

redshift = widgets.FloatSlider(value=SED_PARAMETERS['redshifting']['redshift'],
                               min=0, max=10, step=.1, 
                               description="redshift", continuous_update=False)

l_range = widgets.FloatRangeSlider(value=(1.5, 6), min=1, max=8, step=.1, continuous_update=False)

sliders = widgets.VBox([
    widgets.Text("Fors2 galaxy name"),
    name,
    widgets.Text("Log of adjust factor"),
    log_adjust_factor,
    widgets.Text("Star Formation History"),
    tau_main, age_main, tau_burst, age_burst, f_burst,
    widgets.Text("BC03 SSP"),
    metallicity, 
    widgets.Text("Nebular"),
    logU, f_esc, f_dust, 
    widgets.Text("Dust Attenuation"),
    E_BV_lines, E_BV_factor, uv_bump_amplitude, uv_bump_width, powerlaw_slope, Rv, 
    widgets.Text("IR emission"),
    qpah, umin, alpha, gamma, 
    widgets.Text("Redshift"),
    redshift, 
    widgets.Text("Lambda range (log)"),
    l_range
])



figure = widgets.interactive_output(
    plot_sed,
    {
        'name': name,
        'log_adjust_factor': log_adjust_factor,
        'tau_main': tau_main,
        'age_main': age_main,
        'tau_burst': tau_burst,
        'age_burst': age_burst,
        'f_burst': f_burst,
        'metallicity': metallicity,
        'logU': logU,
        'f_esc': f_esc,
        'f_dust': f_dust,
        'E_BV_lines': E_BV_lines,
        'E_BV_factor': E_BV_factor,
        'uv_bump_amplitude': uv_bump_amplitude,
        'uv_bump_width': uv_bump_width,
        'powerlaw_slope': powerlaw_slope,
        'Rv': Rv,
        'qpah': qpah,
        'umin': umin,
        'alpha': alpha,
        'gamma': gamma,
        'redshift': redshift,
        'l_range': l_range,
    }
)

widgets.HBox([sliders, figure])